# Test classes and functions in FDAASimulationEngine

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from FDAASimulationEngine import SimulationEngine, DemandLocation, Agent
from utils import pre_process_station_name

plt.style.use('dark_background')

In [ ]:
incidents = pd.read_csv("../Data/incidenten_2017.csv", sep=";", decimal=",")
deployments = pd.read_csv("../Data/inzetten_2017.csv", sep=";", decimal=",")
station_locations = pd.read_excel('../Data/kazernepositie en voertuigen.xlsx', sheet_name='adressen')
station_locations['kazerne'] = station_locations['kazerne'].apply(lambda x: pre_process_station_name(x))
from_post_to_coor = pd.read_csv('../Data/nl_postal_codes.csv', encoding = "ISO-8859-1")
from_post_to_coor['Postal Code'] = from_post_to_coor['Postal Code'].apply(lambda x: str(x))
vehicle_allocation = pd.read_excel('../Data/vehicle_allocation.xlsx')

In [ ]:
sim = SimulationEngine(demand_location_definition="postcode_digits", verbose=True)

In [ ]:
sim.fit_parameters(incidents, 
                   deployments, 
                   from_post_to_coor,
                   station_locations,
                   vehicle_allocation,
                   time_of_day_filter=(10, 20))


# Main self variables

In [ ]:
# Variable to manipulate the vehicle status in the simulation
sim.vehicles_status.head()

In [ ]:
# All teh time related parameters are calculated from this DataFrame
sim.merged_log.describe().T

In [ ]:
# Dictinary white different aggreation levels to calculate the vehicle time related parameters
sim.response_time_parameters['level_3']

In [ ]:
# Expected on scene time in minute by event type
sim.parameters_on_scene_time

### Test incident sampling

In [ ]:
sim.initialize_demand_locations()

In [ ]:
incident = sim.generate_incident()
print(incident.start_time)
print(incident.type)
print(incident.priority)
print(incident.required_vehicles)
print(incident.location)
print(list(incident.location_coord))
print(incident.on_secene_duration)

In [ ]:
# agent = Agent()
# sim.set_agent(agent)


In [ ]:
sim.step()

In [ ]:
# N = len(incidents)
# print("{} incidents in 2017.".format(N))

# T = 365*24*60 # minutes

In [ ]:
# sim.verbose = False
# sim.simulate(simulation_time=T)

### Test DemandLocation

In [ ]:
sim.initialize_demand_locations()

In [ ]:
sim.demand_locations["1011"]

In [ ]:
sim.demand_locations["1011"].id

In [ ]:
sim.demand_locations["1011"].incident_type_names

In [ ]:
sim.demand_locations["1011"].incident_type_probs

In [ ]:
sim.demand_locations["1012"].sample_incident_type()

In [ ]:
# check if probabilities match out: 
N = 10000
np.sum(np.array([sim.demand_locations["1011"].sample_incident_type() for i in range(N)]) == "Assistentie Ambulance") / N

In [ ]:
inc_type = sim.demand_locations["1012"].sample_incident_type()
sim.demand_locations["1012"].sample_building_function(inc_type)

## Test deployment

In [ ]:
# Test for the response time
# From the agent
station = 'aalsmeer'
vehicle_type = [11, 12, 13]
#parameters from the incident
incident_priority = 2
incident_location = [52.2538, 4.76889]
on_scene_time = 10288/60

In [ ]:
sim.verbose = False
sim.get_vehicle_simulated_values(incident_priority, incident_location, vehicle_type, on_scene_time)